# 各生物種のスプライスパターンと塩基配列パターンの関連

In [ ]:
# オリジナルモジュールのインポート
from lib.gbkparse import Seq_count
from lib.geneinfo import gene_id

# モジュールのインポート
import logomaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

from plotly.subplots import make_subplots

import requests
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## グラフ描画のための関数定義

In [ ]:
def show_fig(gid, gene_name, specie, intron_file):
    # クラスのインスタンス化
    gbk = Seq_count()

    # gbkファイルの読み込み
    gbk.get_gbk(gid)

    # データの読み込み
    bed = pd.read_csv(intron_file, sep='\t', header=None, usecols=[1,2,5])

    # メジャーなストランドの確認
    major_strand = bed[5].value_counts().idxmax()

    # メジャーなストランドのみを抽出
    bed = bed[bed[5] == major_strand]
    bed = bed.loc[:,[1,2]]

    # 遺伝子領域外のイントロンを除外
    bed = bed[(gbk.start < bed[1]) & (bed[2] < gbk.end)]

    #
    if major_strand == '-':
        bed = bed.apply(lambda x: gbk.end-x)
        bed = bed.loc[:,[2,1]]
        bed.columns = ["start","end"]
        bed = bed.sort_values('start')
    else:
        bed = bed.apply(lambda x: x-gbk.start+1)
        bed.columns = ["start","end"]
        bed = bed.sort_values('start')

    # イントロンの位置の辞書を作成
    intron_dic = {}
    for n,l in enumerate(gbk.intron_list()):
        intron_dic[n+1] = l

    # イントロの位置情報をもとに、イントロン番号を取得
    def start_pos(x):
        for k,v in intron_dic.items():
            if x == v[0]:
                return k
    def end_pos(x):
        for k,v in intron_dic.items():
            if x == v[1]:
                return k
            
    bed['start_pos'] = bed['start'].apply(start_pos)
    bed['end_pos'] = bed['end'].apply(end_pos)
    bed = bed.dropna()
    bed['start_pos'].astype(int)
    bed['end_pos'].astype(int)
    bed['intron_skip'] = bed['end_pos'] - bed['start_pos']
    # 5'側を起点にイントロンの組み合わせの数をカウント
    s = bed.value_counts('start_pos').to_dict()
    for i in range(gbk.intron_num()):
        if i+1 not in s.keys():
            s[i+1] = 0
    sdf = pd.DataFrame.from_dict(s, orient='index').sort_index()


    # 3'側を終点にイントロンの組み合わせの数をカウント
    e = bed.value_counts('end_pos').to_dict()
    for i in range(gbk.intron_num()):
        if i+1 not in e.keys():
            e[i+1] = 0
    edf = pd.DataFrame.from_dict(e, orient='index').sort_index()


    # エクソン配列が3の倍数であるエクソンの表示
    exon_3x = []
    for i in range(gbk.exon_num()):
        if len(gbk.exon_seq(i+1)) % 3 == 0:
            exon_3x.append(1)
        else:
            exon_3x.append(0.1)
    df_3x = pd.DataFrame({"exon":[i+1 for i in range(gbk.exon_num())], 
                    "existence":exon_3x, 
                    "length":[len(gbk.exon_seq(i+1)) for i in range(gbk.exon_num())], 
                    "bar": [1 for i in range(gbk.exon_num())]})


    # イントロンのスプライスバリアントの作成
    skip_bed = bed[bed['intron_skip'] > 1]
    skip_bed = skip_bed.sort_values('intron_skip', ascending=False)
    skip_bed = skip_bed.sort_values('start_pos')

    numlst = [[i,j] for i,j in zip(skip_bed['start_pos'], skip_bed['end_pos'])]
    result = {0:[numlst[0]]}

    for i in numlst[1:]:
        for j in range(len(result.keys())):
            if result[j][-1][1] < i[0]:
                result[j].append(i)
                break
        else:
            result.setdefault(max(result.keys())+1,[i])           
    x_arr = np.array([])
    y_arr = np.array([])
    for i in range(len(result.keys())):
        x_arr = np.concatenate([x_arr, np.array( [i + [None] for i in result[i]]).flatten()])
        y_arr = np.concatenate([y_arr, np.full(3*len(result[i]), i)])

    # # 3'側末端に近いほど高いスコアを与える
    # # 1塩基離れるごとにスコアを1/2にする
    # def match(seq1, seq2):
    #     scores = [(0.9)**i for i in range(len(seq1))]
    #     score = 0
    #     for i,j,k in zip(seq1, seq2, scores):
    #         if i == j:
    #             score += k
    #     return round(score,2)

    # # 3'側末端に近いほど高いスコアを与える
    # # 3'末端に最も高いスコアを与え、1塩基離れるごとにスコアを1減らす
    # def match(seq1, seq2):
    #     scores = [len(seq1)-i for i in range(len(seq1))]
    #     score = 0
    #     for i,j,k in zip(seq1, seq2, scores):
    #         if i == j:
    #             score += k
    #     return round(score,2)

    # 各塩基のマッチングに対して均等にスコアリングする
    def match(seq1, seq2):
        match = 0
        for i,j in zip(seq1, seq2):
            if i == j:
                match += 1
        return round(match/len(seq1),2)

    mtf = "AAATACTAATATCTTTCAAG"
    seq = gbk.gDNA_seq()
    match_result = []
    for i in gbk.intron_list():
        tmp_seq = seq[i[0]:i[1]][-len(mtf):]
        match_result.append(match(mtf, tmp_seq))

    # 各イントロンにおける5'側３塩基パターンが"GTA"のイントロンの分布
    motif_gta = "GTA"
    introns5 = []
    for i in range(gbk.intron_num()):
        if gbk.intron_seq(i)[:len(motif_gta)] == motif_gta:
            introns5.append(1)
        else:
            introns5.append(0)
    df_gta = pd.DataFrame({"intron":[i+1 for i in range(len(introns5))], "existence":introns5, "description": [f"intron: {i+1}" for i in range(len(introns5))]})

    # 各イントロンにおける3'側３塩基パターンが"AAG"のイントロンの分布
    motif_aag= "AAG"
    introns3 = []
    for i in range(gbk.intron_num()):
        if gbk.intron_seq(i)[-len(motif_aag):] == motif_aag:
            introns3.append(1)
        else:
            introns3.append(0)
    df_aag = pd.DataFrame({"intron":[i+1 for i in range(len(introns3))], "existence":introns3, "description": [f"intron: {i+1}" for i in range(len(introns3))]})

    # モチーフをカウントするための関数
    mtf_count = []
    for i in range(gbk.intron_num()):
        loc_count = 0
        seq = gbk.intron_seq(i)
        loc = 0
        while True:
            try:
                loc = seq.index("TCTT",loc)
                loc += 1
                loc_count += 1
            except ValueError:
                break
        mtf_count.append(loc_count)

    # グラフの描画
    threshold = 0.5
    titles = ["3の倍数の塩基数のエクソン", "スプライスされたイントロンの起点の数", "スプライスされたイントロンの長さ", "スプライスされたイントロンの終点の数",f"3'側イントロン末端配列と{mtf}配列とのマッチングスコア",
            "5'側末端の塩基配列がGTAのイントロン","3'側末端の塩基配列がAAGのイントロン", f"各イントロンの'TCTT'の数"]
    fig = make_subplots(rows=8, cols=1, shared_xaxes=True, vertical_spacing=0.05, subplot_titles=titles)
    fig.add_trace(go.Bar(x=df_3x['exon'], y=df_3x['existence'], customdata=df_3x['length'], hovertemplate="Intron%{x}<br>Lenght: %{customdata}<extra></extra>"), row=1, col=1)
    fig.add_trace(go.Bar(x=sdf.index, y=sdf[0], hovertemplate='Intron%{x}<extra></extra>'), row=2, col=1)
    fig.add_trace(go.Scatter(x=x_arr, y=y_arr, hovertemplate='Intron edge: Intron%{x}<extra></extra>'), row=3, col=1)
    fig.add_trace(go.Bar(x=edf.index, y=edf[0], hovertemplate='Intron%{x}<extra></extra>'), row=4, col=1)
    fig.add_trace(go.Bar(x=[i+1 for i in range(len(match_result))], y=match_result, hovertemplate='Intron%{x}<br>Match: %{y}<extra></extra>'), row=5, col=1)
    fig.add_trace(go.Scatter(x=[1,gbk.intron_num()], y=[threshold, threshold],line=dict(color='red', dash='dot')), row=5, col=1)
    fig.add_trace(go.Bar(x=df_gta['intron'], y=df_gta['existence'], hovertemplate='Intron: %{x}<extra></extra>'), row=6, col=1)
    fig.add_trace(go.Bar(x=df_aag['intron'], y=df_aag['existence'], hovertemplate='Intron: %{x}<extra></extra>'), row=7, col=1)
    fig.add_trace(go.Bar(x=[i+1 for i in range(gbk.intron_num())], y=mtf_count, hovertemplate= 'Intron: %{x}<br>TCTT count: %{y}<extra></extra>'), row=8, col=1)

    # 各サブプロットのタイトルを追加
    fig.update_layout(width=1500, height=800, title_text=f"{specie} {gene_name}遺伝子 (NM_001267550.2)", showlegend=False)
    # サブプロットタイトルの位置、フォントサイズの変更
    for i in range(len(fig.layout.annotations)):
        fig.layout.annotations[i].font.size = 12
        fig.layout.annotations[i].x = 0
        fig.layout.annotations[i].xanchor = 'left'
    fig.show()


In [ ]:
# ヒトTTN遺伝子の可視化
gene_name = 'TTN'
specie = 'human'
intron_file = f'../data/intron/{specie}_{gene_name.lower()}_intron.tsv'
gid = "7273"
show_fig(gid, gene_name, specie, intron_file)

In [ ]:
# チンパンジーTTN遺伝子の可視化
gene_name = 'TTN'
specie = 'chimp'
intron_file = f'../data/intron/{specie}_{gene_name.lower()}_intron.tsv'
gid = "459784"
show_fig(gid, gene_name, specie, intron_file)

In [ ]:
# アカゲザルTTN遺伝子の可視化
gene_name = 'TTN'
specie = 'macaca'
intron_file = f'../data/intron/{specie}_{gene_name.lower()}_intron.tsv'
gid = "703527"
show_fig(gid, gene_name, specie, intron_file)

In [ ]:
# ウサギTTN遺伝子の可視化
gene_name = 'TTN'
specie = 'rabbit'
intron_file = f'../data/intron/{specie}_{gene_name.lower()}_intron.tsv'
gid = "100008570"
show_fig(gid, gene_name, specie, intron_file)

In [ ]:
# ラットTTN遺伝子の可視化
gene_name = 'TTN'
specie = 'rat'
intron_file = f'../data/intron/{specie}_{gene_name.lower()}_intron.tsv'
gid = "84015"
show_fig(gid, gene_name, specie, intron_file)

In [ ]:
# マウスTTN遺伝子の可視化
gene_name = 'TTN'
specie = 'mouse'
intron_file = f'../data/intron/{specie}_{gene_name.lower()}_intron.tsv'
gid = "22138"
show_fig(gid, gene_name, specie, intron_file)

In [ ]:
# チキンTTN遺伝子の可視化
gene_name = 'TTN'
specie = 'chicken'
intron_file = f'../data/intron/{specie}_{gene_name.lower()}_intron.tsv'
gid = "424126"
show_fig(gid, gene_name, specie, intron_file)